In [ ]:
import itertools
import os.path as path

import aesara_theano_fallback.tensor as tt
import astropy.io.fits as fits
import astropy.timeseries as timeseries
import astropy.units as u
import arviz as az
import celerite2
import celerite2.terms as terms
import celerite2.theano.terms as theano_terms
import corner
import exoplanet as xo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import pymc3_ext as pmx
import scipy.stats as stats

# import third_party.keplersplinev2.keplersplinev2 as ksp

import adhocfitter.astro as astro
import adhocfitter.plotting as plotting
import adhocfitter.priors as priors
import adhocfitter.timeseries as aftimeseries

# Disable OpenMP parallelism
import os
os.environ["OMP_NUM_THREADS"] = "1"

DATA_DIR = '../data'

In [ ]:
test_epoch = np.array([2110.06588, 1855.2442])  # TJD
test_epoch_unc = np.array([0.00028, 0.0022])
test_period = np.array([9.127055, 3.09833])  # day
test_period_unc = np.array([0.0000073, 0.000021])  #np.array([0.00001, 0.000023])
test_last_seen = np.array([2111.0, 1855.3])
# 2073.55739092,  2073.55767828,  2073.55796009]),
#  array([ 1907.91526705,  1907.91617858,  1907.91709297])
test_epoch = (test_last_seen - test_epoch) // test_period * test_period + test_epoch
print(test_epoch)
test_duration = np.array([4.9, 2.0])  # hour
test_duration_unc = np.array([6., 4.])
test_omega_vec = np.array([0., 1.])

test_radius_planet = np.array([0.066, 0.022])  # Stellar radius
test_radius_planet_unc = np.array([0.5, 0.5])
test_impact_param = np.array([0.615, 0.756])
test_limb_dark = np.array([0.37, 0.21])

# Estimated values from previous fits
test_mass_star = 1.086714  # Joint model posterior mean.
test_mass_star_unc = 0.05258207  # Joint model posterior standard deviation.
test_feh = 0.415
test_eep = 380
# A_V extinction
test_av = 0.15  # From Keivan.
test_av_upper = 0.70  # Upper limit from Schlegel 1998 dust map.
# Gaia EDR3 parallax
PARALLAX_GAIA_EDR3 = 5.76044602669829
PARALLAX_GAIA_EDR3_UNC= 0.01048182
# Gaia EDR3 parallax zero-point correction, computed by:
# zpt.get_zpt(
#     phot_g_mean_mag=10.8542,
#     nu_eff_used_in_astrometry=1.5263987,
#     pseudocolour=float('nan'),
#     ecl_lat=-68.38768994372302,
#     astrometric_params_solved=31,  # 5-param solution.
# )
PARALLAX_GAIA_EDR3 -= -0.012446

test_mass_planet = (np.array([75.3, 10.2]) * u.M_earth).to(u.M_sun).value / test_mass_star
test_mass_planet_unc = (np.array([300., 100.]) * u.M_earth).to(u.M_sun).value / test_mass_star # Upper limit.

print(test_mass_planet, test_mass_planet_unc)

# TIC 8.1 values
# test_density_star = 0.839021  # g cm^-3
# test_density_star_unc = 0.190382
# test_radius_star = 1.18093  # R_sun
# test_radius_star_unc = 0.0597739
# test_mass_star = 0.98  # M_sun
# test_mass_star_unc = 0.131459

# FEROS values
# test_density_star = 1.052  # g cm^-3
# test_density_star_unc = 0.052
# test_radius_star = 1.104  # R_sun
# test_radius_star_unc = 0.011
# test_mass_star = 1.001  # M_sun
# test_mass_star_unc = 0.026

# print(test_mass_star/test_radius_star**3 * astro.SOLAR_DENSITY*1e-3)

test_params = {
    "t0": test_epoch,
    "period": test_period,
    "rp": test_radius_planet,
    "b": test_impact_param,
    "ecc_fix": np.array([False, True]),
    "ecc": np.array([0., 0.]),
    "omega": np.array([np.pi/2, np.pi/2]),
    "u": {"tess": test_limb_dark, "Rc": test_limb_dark, "tess_c": test_limb_dark},
#     "rho_star": test_density_star,
    "m_star": test_mass_star,
#     "r_star": test_radius_star,
    "tdur": test_duration,
    "m_planet": test_mass_planet,
    "av": test_av,
    "parallax": PARALLAX_GAIA_EDR3,
    "feh": test_feh,
    "eep": test_eep,
}

prior_unc = {
    "t0": test_epoch_unc,
    "period": test_period_unc,
    "rp": test_radius_planet_unc,
#     "rho_star": test_density_star_unc,
    "m_star": test_mass_star_unc,
#     "r_star": test_radius_star_unc,
    "m_planet": test_mass_planet_unc,
    "tdur": test_duration_unc,
    "av": test_av_upper,
    "parallax": PARALLAX_GAIA_EDR3_UNC,
    "albedo_bond": 0.7,
}

In [ ]:
def add_normal_prior(name, test_params, prior_unc, shape=None):
    if shape is None:
        return pm.Normal(
            name, mu=test_params[name], sd=prior_unc[name])
    else:
        return pm.Normal(
            name, mu=test_params[name], sd=prior_unc[name], shape=shape)

def add_uniform_prior(name, test_params, prior_unc, shape=None):
    lower = test_params[name] - prior_unc[name]
    upper = test_params[name] + prior_unc[name]
    if shape is None:
        return pm.Uniform(
            name, lower=lower, upper=upper, testval=test_params[name])
    else:
        return pm.Uniform(
            name, lower=lower, upper=upper, testval=test_params[name], shape=shape)

In [ ]:
def optimize_model(model, passes=None):
    with model:
        if passes is None:
            return pmx.optimize(start=model.test_point)
        map_soln = pmx.optimize(start=model.test_point, vars=[model[v] for v in passes[0]])
        for p in passes[1:]:
            map_soln = pmx.optimize(start=map_soln, vars=[model[v] for v in p])
        map_soln = pmx.optimize(start=map_soln)
    return map_soln

In [ ]:
rv_table = aftimeseries.read_generic_rv(path.join(DATA_DIR, 'toi_2000_table_08.csv'))
chiron_time, chiron_rv, chiron_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'CHIRON')
feros_time, feros_rv, feros_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'FEROS')
harps_time, harps_rv, harps_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'HARPS')

rv_names = ['chiron', 'feros', 'harps']
rv_times = [chiron_time, feros_time, harps_time]
rv_data = [chiron_rv, feros_rv, harps_rv]
rv_uncs = [chiron_rv_unc, feros_rv_unc, harps_rv_unc]
num_rv_outside = len(rv_data)

rv_times_all = np.concatenate(rv_times)
rv_data_all = np.concatenate(rv_data)
rv_uncs_all = np.concatenate(rv_uncs)

test_gamma = np.array([np.average(i) for i in rv_data])
test_rv_unc = np.array([np.average(i) for i in rv_uncs])
test_jitter = np.array([1e-3]*len(rv_data))

test_params['K'] = np.array([23., 9.])
prior_unc['K'] = np.array([50., 30.])
test_params['rv_gamma'] = test_gamma
prior_unc['rv_gamma'] = np.array([1000.]*num_rv_outside)
test_params['rv_jitter'] = test_jitter
prior_unc['rv_jitter'] = np.array([15., 30., 15.])

test_params['rv_fit_planet'] = np.array([True, True])

test_gamma, test_rv_unc, test_jitter, prior_unc['rv_gamma'], prior_unc['rv_jitter']

In [ ]:
activity_index = ['fwhm', 'bis_span', 's_mw', 'rhk']
fig, axs = plt.subplots(
    len(activity_index),
    figsize=(6, 4*len(activity_index)),
    dpi=144,
    sharex=True,
)
false_alarm_prob = [0.1, 0.05, 0.01]
for idx, ax in zip(activity_index, axs):
    ls_obj = timeseries.LombScargle(harps_df['jdb'], harps_df[idx])
    freq, power = ls_obj.autopower(
        minimum_frequency=1/100.,
        maximum_frequency=2.,
    )
    ax.plot(1/freq, power)
    false_alarm_levels = ls_obj.false_alarm_level(false_alarm_prob)
    for fal in false_alarm_levels:
        ax.axhline(fal, linestyle='--', color='gray')
    for p in [90, 17.5]:
        ax.axvline(p, linestyle='--', color='gray')
    ax.set_xscale('log')
    ax.text(0.05, 0.9, idx.upper(), fontsize='large', transform=ax.transAxes)
    ax.tick_params(which='both', direction='in', top=True, right=True)
    ax.set_ylabel('Normalized power')
ax.set_xlabel('Period (day)')
fig.tight_layout()
fig.savefig('../plots/toi_2000_harps_rv_activity_periodograms.png', bbox_inches='tight')

In [ ]:
ls_bis_span = timeseries.LombScargle(
    rv_times_all, np.ones_like(rv_times_all), center_data=False, fit_mean=False)
wn_freq, wn_pow = ls_bis_span.autopower(
    minimum_frequency=1/1000.,
    maximum_frequency=3.,
)
fig, ax = plt.subplots(dpi=144)
ax.plot(1/wn_freq, wn_pow)
ax.set_xscale('log')
ax.set_xlabel('Period (day)')
ax.set_ylabel('Spectral window function')
false_alarm_levels = ls_obj.false_alarm_level(false_alarm_prob)
for fal in false_alarm_levels:
    ax.axhline(fal, linestyle='--', color='gray')
for p in [90, 17.5]:
    ax.axvline(p, linestyle='--', color='gray')

In [ ]:
ls_bis_span = timeseries.LombScargle(np.array(harps_df.jdb), np.array(harps_df.bis_span))
plt.plot(*ls_bis_span.autopower())

In [ ]:
test_params.update({
    'period_rv_only': np.array([89.9, 17.2]),
    't0_rv_only': np.array([2174.56, 2212.0]),
    'K_rv_only': np.array([15.5, 6.]),
})
prior_unc.update({
    'period_rv_only': np.array([20.]),
    'K_rv_only': np.array([30., 15.]),
})
num_planet_rv_only = 2

In [ ]:
def _add_fixed_eccentricity(fix_flag):
    if all(fix_flag):
        return None, [None, None]
    if not any(fix_flag):
        ecc_vec = pmx.UnitDisk(f'sqrt_ecc_vec', testval=np.array([1e-6, 1e-6]), shape=(2, len(fix_flag)))
        ecc = pm.Deterministic('ecc', tt.sum(ecc_vec*ecc_vec, axis=0))
        omega_vec = ecc_vec / tt.sqrt(ecc)
        omega = pm.Deterministic('omega', tt.arctan2(omega_vec[1], omega_vec[0]))
        return ecc, omega_vec
    ecc_stack = []
    omega_vec_stack = []
    for i, ecc_fix in enumerate(fix_flag):
        if ecc_fix:
            ecc_stack.append(tt.as_tensor_variable(0.))
            omega_vec_stack.append(tt.as_tensor_variable(np.array([0., 1.])))
        else:
            ecc_vec = pmx.UnitDisk(f'sqrt_ecc_vec_{i}', testval=np.array([1e-6, 1e-6]))
            ecc = tt.sum(ecc_vec*ecc_vec, axis=0)
            ecc_stack.append(ecc)
            omega_vec_stack.append(ecc_vec / tt.sqrt(ecc))
    ecc = pm.Deterministic('ecc', tt.stack(ecc_stack, axis=0))
    omega_vec = tt.stack(omega_vec_stack, axis=1)
    omega = pm.Deterministic('omega', tt.arctan2(omega_vec[1], omega_vec[0]))
    return ecc, omega_vec

In [ ]:
def make_rv_model_with_rv_only_planets(
    num_planets, ecc_flag, num_planets_rv_only,
    rv_times, rv_data, rv_uncs,
    test_params, prior_unc):
    with pm.Model() as model:
        period = pm.Normal('period', mu=test_params['period'], sigma=prior_unc['period'], shape=num_planets)
        epoch = pm.Normal('t0', mu=test_params['t0'], sigma=prior_unc['t0'], shape=num_planets)
        rv_semiamp = pm.Normal('K', mu=test_params['K'], sigma=prior_unc['K'], shape=num_planets)
        
        ecc, omega_vec = _add_fixed_eccentricity(ecc_flag)

        orbit = xo.orbits.KeplerianOrbit(
            period=period,
            t0=epoch,
            # b=impact_param,
            ecc=ecc,
            cos_omega=omega_vec[0],
            sin_omega=omega_vec[1],
            # r_star=radius_star,
        )

        period_rv_only = add_uniform_prior('period_rv_only', test_params, prior_unc, shape=num_planets_rv_only)
        rv_semiamp_rv_only = add_uniform_prior('K_rv_only', test_params, prior_unc, shape=num_planets_rv_only)
        epoch_rv_only = pm.Uniform(
            't0_rv_only', lower=test_params['t0_rv_only']-period_rv_only/2,
            upper=test_params['t0_rv_only']+period_rv_only/2,
            testval=test_params['t0_rv_only'],
            shape=num_planets_rv_only)
        orbit_rv_only = xo.orbits.KeplerianOrbit(
            period=period_rv_only,
            t0=epoch_rv_only,
        )

        num_rvs = len(rv_data)
        pm.Normal('rv_gamma', mu=test_params['rv_gamma'], sigma=prior_unc['rv_gamma'], shape=num_rvs)
        pm.HalfNormal(
            'rv_jitter', sigma=prior_unc["rv_jitter"], shape=num_rvs)
        
#         model_set_up_polynomial_detrend(model, rv_times, 2)
        
#         trend_func = trend_generator
#         if trend_func is None:
#             def trend_func(model, num_rv):
#                 for i in range(num_rv):
#                     yield 0.

        rv_residuals = []
        for i, (rv_time, rv, rv_unc) in enumerate(zip(
            rv_times, rv_data, rv_uncs)): #, trend_func(model, num_rvs))):
            rv_predict = pm.Deterministic(f'rv_pred_{i}', orbit.get_radial_velocity(rv_time, K=model.K))
            rv_only_pred = pm.Deterministic(
                f'rv_only_pred_{i}',
                orbit_rv_only.get_radial_velocity(rv_time, K=rv_semiamp_rv_only))
            rv_predict_sum = (
                tt.sum(rv_predict, axis=1)
                + (tt.sum(rv_only_pred, axis=1) if num_planets_rv_only > 1 else rv_only_pred)
                + model.rv_gamma[i] # + trend
            )

            rv_residuals.append(rv - rv_predict_sum)

            total_unc = tt.sqrt(rv_unc * rv_unc + model.rv_jitter[i] * model.rv_jitter[i])
            obs_likelihood = pm.Normal(
                f"rv_obs_{i}",
                mu=rv_predict_sum,
                sd=total_unc,
                observed=rv,
            )

        # model_set_up_gp_sho(model, rv_times, rv_residuals, rv_uncs, model.rv_jitter)

        pm.Deterministic('log_prob', model.logpt)
    return model

In [ ]:
rv_model = make_rv_model_with_rv_only_planets(
    2, [False, True], 2, rv_times, rv_data, rv_uncs, test_params, prior_unc)

rv_map_soln = optimize_model(
    rv_model,
    [
        ['rv_jitter', 'rv_gamma', 'K', 'K_rv_only'],  #, 'gp_sigma', 'gp_rho', 'gp_tau'],#, 'rv_trend_coeff'],
        ['t0', 'period', 't0_rv_only', 'period_rv_only'],
        # ['gp_sigma', 'gp_rho', 'gp_tau'],
    ]
)
rv_map_soln

In [ ]:
fig, rv_folded_axs, rv_unfolded_ax, rv_residual_ax = plotting.make_multi_planet_rv_axes(4)
rv_unfolded_ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_ylabel('Residuals ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_xlabel('$\mathrm{BJD}_\mathrm{TDB} - 2{,}457{,}000$')
rv_folded_axs[0].set_title('c')
rv_folded_axs[1].set_title('b')
for ax in rv_folded_axs:
    ax.set_xlabel('Phase')
    ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')

plot_orbit = xo.orbits.KeplerianOrbit(
    **{k: rv_map_soln[k] for k in ['period', 't0', 'ecc', 'omega']})

plot_orbit = xo.orbits.KeplerianOrbit(
    period=np.concatenate([rv_map_soln['period'], rv_map_soln['period_rv_only']]),
    t0=np.concatenate([rv_map_soln['t0'], rv_map_soln['t0_rv_only']]),
    ecc=np.concatenate([rv_map_soln['ecc'], [0.]*num_planet_rv_only]),
    omega=np.concatenate([rv_map_soln['omega'], [np.pi/2]*num_planet_rv_only]),
)

plot_rv_data_style = {
    'markeredgecolor': '#2a0944',
    'markeredgewidth': 0.5,
    'alpha': 0.7,
    'markersize': 3,
    'ecolor': 'gray',
    'elinewidth': 1,
}
    
plot_rv_inst_styles = [
    {'color': '#3b185f', 'fmt': '^'},
    {'color': '#a12568', 'fmt': 'd'},
    {'color': '#fec260', 'fmt': 'o'},
]

# rv_trend_func, rv_trend_unc_func = gp_trend_func_generator(
#     gp_sho, rv_map_soln, rv_times, rv_data, rv_uncs, return_unc=True)

plotting.plot_multi_planet_folded_rv(
    rv_folded_axs, rv_unfolded_ax, rv_residual_ax,
    4, plot_orbit, np.concatenate([rv_map_soln['K'], rv_map_soln['K_rv_only']]),
    rv_map_soln['rv_gamma'],
    rv_map_soln['rv_jitter'],
    rv_times,
    rv_data,
    rv_uncs,
    ['CHIRON', 'FEROS', 'HARPS'],
    # trends=trend_generator(rv_map_soln, len(rv_data)),
    # trends=gp_generator(rv_map_soln, len(rv_data)),
    # model_trend_func=rv_trend_func,
    # model_trend_unc_func=rv_trend_unc_func,
    # model_trend_func=gp_trend_func_generator(gp_sho, rv_map_soln, rv_times, rv_data, rv_uncs),
    # model_trend_func=lambda t: polynomial_design_matrix(t, rv_map_soln['rv_time_offset'], 2) @ rv_map_soln['rv_trend_coeff'],
    rv_data_style=plot_rv_data_style,
    rv_inst_styles=plot_rv_inst_styles,
)

rv_unfolded_ax.set_xlim(1905, 2375)
rv_unfolded_ax.legend(loc='lower center')

for ax, p in zip(rv_folded_axs, rv_map_soln["period"]):
    ax.text(0.05, 0.05, f'$P = {p:.6f}\,\mathrm{{d}}$', transform=ax.transAxes)


In [ ]:
rv_traces = []

In [ ]:
with rv_model:
    trace = pmx.sample(
        tune=5000,
        draws=5000,
        start=rv_map_soln,
        cores=32,
        chains=32,
        # initial_accept=0.5,
        target_accept=0.97,
        return_inferencedata=True,
        idata_kwargs={
            'log_likelihood': False,
        },
#         parameter_groups=[
#             pmx.ParameterGroup([simple_model.period, simple_model.t0]),
#             pmx.ParameterGroup([simple_model.tdur, simple_model.rp, simple_model.b]),
#             pmx.ParameterGroup([simple_model.mean_flux, simple_model.lc_jitter]),
#         ],
    )
rv_traces.append(trace)

In [ ]:
trace['posterior']

In [ ]:
trace.to_netcdf(
    '../chains/toi2000_rv_only_90d_17d.nc')
    # groups=['posterior', 'log_likelihood', 'sample_stats'])

In [ ]:
trace = az.from_netcdf('../chains/toi2000_rv_only_90d_17d.nc')
trace_3p = az.from_netcdf('../chains/toi2000_rv_only_90d.nc')

In [ ]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
az.plot_trace(trace, var_names=display_var_names)
plt.tight_layout()

In [94]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0', 'ecc', 'omega',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
summary_3p = az.summary(
    trace_3p, var_names=display_var_names,
    round_to=7,
    hdi_prob=0.997,
    skipna=True,
    # kind='stats',
    # stat_funcs={
    #     median
    # }
    coords={"ecc_dim_0": [0], "omega_dim_0": [0]},
    circ_var_names={'omega'},
    stat_funcs={
        'median': np.median,
        '-': lambda x: np.quantile(x, 0.16) - np.median(x),
        '+': lambda x: np.quantile(x, 0.84) - np.median(x),
    },
)
summary_3p

/opt/conda/lib/python3.9/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/opt/conda/lib/python3.9/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_0.15%,hdi_99.85%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,median,-,+
period[0],9.127056,0.000100,9.126761e+00,9.127354,2.000000e-07,1.000000e-07,262225.955838,117714.594080,1.000280,9.127056,-0.000100,0.000100
period[1],3.098290,0.000197,3.097719e+00,3.098874,4.000000e-07,3.000000e-07,257859.466112,121590.794132,1.000218,3.098290,-0.000195,0.000196
t0[0],2110.065875,0.004998,2.110051e+03,2110.080903,9.800000e-06,6.900000e-06,262318.129617,121999.444219,1.000362,2110.065872,-0.004971,0.004973
t0[1],1855.241018,0.019753,1.855182e+03,1855.299321,3.870000e-05,2.740000e-05,260601.344009,122573.739122,1.000334,1855.240985,-0.019635,0.019674
sqrt_ecc_vec_0[0],-0.184189,0.081018,-3.603775e-01,0.120038,2.345000e-04,1.659000e-04,149831.971336,85019.078465,1.000097,-0.195730,-0.064607,0.087207
sqrt_ecc_vec_0[1],-0.060607,0.167664,-4.487414e-01,0.377508,4.680000e-04,3.309000e-04,132601.205648,136575.963603,1.000104,-0.070283,-0.166880,0.193103
ecc[0],0.072274,0.037105,3.000000e-07,0.208030,9.640000e-05,7.060000e-05,146504.093423,101417.132021,1.000040,0.069299,-0.033080,0.037591
ecc[1],0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,160000.000000,160000.000000,NaN,0.000000,0.000000,0.000000
omega[0],-2.869136,0.784226,3.811322e-01,-0.214577,3.023500e-03,5.220200e-03,138142.126792,152059.054436,1.000045,-2.082607,-0.681459,4.764533
omega[1],1.570796,-0.000000,1.570796e+00,1.570796,-0.000000e+00,0.000000e+00,160000.000000,160000.000000,NaN,1.570796,0.000000,0.000000


In [95]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0', 'ecc', 'omega',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
summary = az.summary(
    trace, var_names=display_var_names,
    round_to=7,
    hdi_prob=0.997,
    skipna=True,
    # kind='stats',
    # stat_funcs={
    #     median
    # }
    coords={"ecc_dim_0": [0], "omega_dim_0": [0]},
    circ_var_names={'omega'},
    stat_funcs={
        'median': np.median,
        '-': lambda x: np.quantile(x, 0.16) - np.median(x),
        '+': lambda x: np.quantile(x, 0.84) - np.median(x),
    },
)
summary

/opt/conda/lib/python3.9/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/opt/conda/lib/python3.9/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_0.15%,hdi_99.85%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,median,-,+
period[0],9.127056,0.000100,9.126766,9.127364,2.000000e-07,1.000000e-07,243749.305372,122461.135197,1.000234,9.127056,-0.000099,0.000099
period[1],3.098302,0.000189,3.097733,3.098856,4.000000e-07,3.000000e-07,234454.822184,123894.147924,1.000234,3.098302,-0.000188,0.000188
t0[0],2110.065927,0.004993,2110.050956,2110.080590,1.020000e-05,7.200000e-06,240825.593138,123000.694298,1.000112,2110.065930,-0.004957,0.004966
t0[1],1855.242212,0.019475,1855.183172,1855.297992,4.030000e-05,2.850000e-05,233796.561716,123893.727777,1.000166,1855.242223,-0.019384,0.019331
sqrt_ecc_vec_0[0],-0.169048,0.062888,-0.325116,0.077174,1.835000e-04,1.297000e-04,140690.262430,77700.766452,1.000086,-0.175135,-0.054151,0.064783
sqrt_ecc_vec_0[1],-0.129944,0.134262,-0.430034,0.261885,4.193000e-04,2.965000e-04,114475.919565,104037.982415,1.000132,-0.148567,-0.114560,0.162061
ecc[0],0.067444,0.032400,0.000006,0.181892,8.480000e-05,6.000000e-05,133590.064694,92895.048559,1.000030,0.063981,-0.027412,0.034420
ecc[1],0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,160000.000000,160000.000000,NaN,0.000000,0.000000,0.000000
omega[0],-2.550544,0.614270,0.785852,-0.645068,9.644000e-04,4.564900e-03,123200.114140,149058.923817,1.000095,-2.218772,-0.504687,4.336813
omega[1],1.570796,-0.000000,1.570796,1.570796,-0.000000e+00,0.000000e+00,160000.000000,160000.000000,NaN,1.570796,0.000000,0.000000


In [ ]:
rv_flat_samples = trace.posterior.stack(sample=("chain", "draw"))
rv_median_soln = {k:v.data for k, v in rv_flat_samples.median(dim='sample').items()}
rv_max_post_index = rv_flat_samples.log_prob.argmax(dim='sample')
rv_max_post_soln = {k:v.data for k, v in rv_flat_samples[{'sample': rv_max_post_index}].items()}

In [ ]:
astro.calculate_min_planet_mass_earth(
    np.array([22.807773, 5.948806, 15.426891, 6.332211]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127056, 3.098302, 90.735801, 17.289302]),
    1.082411,
)

In [ ]:
astro.calculate_min_planet_mass_earth(
    np.array([23.7, 4.59, 15.426891, 6.332211]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127055, 3.098302, 90.735801, 17.289302]),
    1.082411,
)

In [ ]:
astro.calculate_rv_semiamp(
    np.array([75.3, 10.2]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127055, 3.09833]),
    1.082411,
)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        np.array(rv_flat_samples['ecc']),
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
(14.3-11)/2.6, (11-14.3)/14.3, (78.3-81.7)/4.7

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        0.,
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.vstack([
            np.random.normal(23.7, 1.0, random_size[1]),
            np.random.normal(4.59, 1.0, random_size[1]),
        ]),
        0.,
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K_rv_only']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K_rv_only']),
        0.,
        np.array(rv_flat_samples['period_rv_only']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
104.30264212 - 114.13412687, 124.37240489 - 114.13412687

In [ ]:
31.77303186-26.93358525, 22.11476994-26.93358525

In [ ]:
np.quantile(
    (astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        np.array(rv_flat_samples['ecc']),
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ) /
    np.sin(np.vstack([
        np.random.normal(87.94, 0.13, random_size[1]),
        np.random.normal(84.73, 0.5, random_size[1])])/180*np.pi)),
    np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
np.array([73.76581984, 83.03444882]) - 78.33842635, np.array([11.80104626, 16.90607814]) - 14.33273728

In [ ]:
def rv_bic(rv_soln, rv_vals, rv_uncs, num_params):
    num_points = 0
    log_likelihood = 0
    for i, (my_rv, my_unc, my_jitter) in enumerate(zip(rv_vals, rv_uncs, rv_soln['rv_jitter'])):
        num_points += len(my_rv)
        log_likelihood += priors.log_prob_gaussian(
            rv_soln[f'rv_pred_{i}'],# + rv_soln[f'rv_gp_pred_{i}'],
            my_rv,
            my_unc*my_unc + my_jitter*my_jitter)
    return num_params * np.log(num_points) - 2 * log_likelihood

In [ ]:
rv_bic(rv_max_post_soln, rv_data, rv_uncs, 19)

In [ ]:
rv_bic(rv_max_post_soln, rv_data, rv_uncs, 16)

In [ ]:
600.0033015661337-513.21787135048135

In [ ]:
rv_max_post_soln

In [ ]:
rv_flat_samples.quantile([1-0.999999426696856, 1-0.999993204653751, 1-0.999936657516334], dim='sample')

In [ ]:
rv_max_post_soln['period_rv_only']

In [ ]:
fig, rv_folded_axs, rv_unfolded_ax, rv_residual_ax = make_multi_planet_rv_axes(2)
rv_unfolded_ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_ylabel('Residuals ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_xlabel('$\mathrm{BJD}_\mathrm{TDB} - 2{,}457{,}000$')
rv_folded_axs[0].set_title('b')
rv_folded_axs[1].set_title('c')
for ax in rv_folded_axs:
    ax.set_xlabel('Phase')
    ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')

plot_orbit = xo.orbits.KeplerianOrbit(
    **{k: rv_max_post_soln[k] for k in ['period', 't0', 'ecc', 'omega']})

plot_rv_data_style = {
    'markeredgecolor': '#2a0944',
    'markeredgewidth': 0.5,
    'alpha': 0.7,
    'markersize': 3,
    'ecolor': 'gray',
    'elinewidth': 1,
}
    
plot_rv_inst_styles = [
    {'color': '#3b185f', 'fmt': '^'},
    {'color': '#a12568', 'fmt': 'd'},
    {'color': '#fec260', 'fmt': 'o'},
]

rv_trend_func, rv_trend_unc_func = gp_trend_func_generator(
    gp_sho, rv_max_post_soln, rv_times, rv_data, rv_uncs, return_unc=True)

plot_multi_planet_folded_rv(
    rv_folded_axs[::-1], rv_unfolded_ax, rv_residual_ax,
    2, plot_orbit, rv_max_post_soln['K'],
    rv_max_post_soln['rv_gamma'],
    rv_max_post_soln['rv_jitter'],
    rv_times,
    rv_data,
    rv_uncs,
    ['CHIRON', 'FEROS', 'HARPS'],
    # trends=trend_generator(rv_max_post_soln, len(rv_data)),
    trends=gp_generator(rv_max_post_soln, len(rv_data)),
    model_trend_func=rv_trend_func,
    model_trend_unc_func=rv_trend_unc_func,
    # model_trend_func=gp_trend_func_generator(rv_max_post_soln, rv_times, rv_data, rv_uncs),
    # model_trend_func=lambda t: polynomial_design_matrix(t, rv_max_post_soln['rv_time_offset'], 2) @ rv_max_post_soln['rv_trend_coeff'],
    rv_data_style=plot_rv_data_style,
    rv_inst_styles=plot_rv_inst_styles,
)

rv_unfolded_ax.set_xlim(1905, 2375)
rv_unfolded_ax.legend(loc='lower center')

for ax, p in zip(rv_folded_axs[::-1], rv_max_post_soln["period"]):
    ax.text(0.05, 0.05, f'$P = {p:.6f}\,\mathrm{{d}}$', transform=ax.transAxes)

fig.savefig('plot/toi2000_rv_only_gp_sho.pdf', bbox_inches='tight')

In [ ]:
display_var_names = [
    'period', 't0', 'sqrt_ecc_vec_0', 'K',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho',
]
_ = corner.corner(
    trace,
    quantiles=(0.16, 0.5, 0.84),
    levels=1-np.exp(-np.array([0.5, 1, 1.5, 2, 2.5])**2/2),
    var_names=display_var_names,
)